# Working with trajectories

Some of the methods in `CoherentStructures.jl` currently work with velocity fields only.
However, there are also some methods that work on trajectories directly. These include
the graph Laplacian based methods and the Transfer-Operator based methods for approximating the
dynamic Laplacian.

We show here how to use some of these methods.

We first generate some trajectories on a set of `n' random points for the double gyre.

In [ ]:
using CoherentStructures,Tensors

n = 500
tspan=range(0,stop=1.0, length=20)
initial_points = [Vec{2}(rand(2)) for i in 1:n]
trajectories = [
    flow(rot_double_gyre, initial_points[i],tspan) for i in 1:n
    ]
# ## FEM adaptive TO method
# Generate a triangulation
# If this call fails or does not return the initial points may not be unique.
ctx, _ = irregularDelaunayGrid(initial_points)

#Generate stiffness and mass matrices, solve eigenproblem.

If this call fails or does not return the initial points may not be unique.

In [ ]:
S = adaptiveTOCollocationStiffnessMatrix(ctx,(i,ts) -> trajectories[i], tspan; flow_map_mode=1
    )
M = assembleMassMatrix(ctx)

using Arpack
λ, V = eigs(S,M, which=:SM,nev=6)

We can plot the spectrum obtained.

In [ ]:
import Plots
plot_real_spectrum(λ)

K-means clustering yields the coherent vortices.

In [ ]:
using Clustering
function iterated_kmeans(numiterations, args...)
    best = kmeans(args...)
    for i in 1:(numiterations - 1)
        cur = kmeans(args...)
        if cur.totalcost < best.totalcost
            best = cur
        end
    end
    return best
end

n_partition = 3
res = iterated_kmeans(20, permutedims(V[:,2:n_partition]), n_partition)
u = kmeansresult2LCS(res)
u_combined = sum([u[:,i] * i for i in 1:n_partition])

#We plot the result. The result looks "messy" due to the

fact that we used quite few trajectories

In [ ]:
fig = plot_u(ctx, u_combined, 400, 400;
    color=:viridis, colorbar=:none, title="$n_partition-partition of double gyre")

Plots.plot(fig)

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*